In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "00_efficientnet-b2"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 8
    model_name = 'efficientnet-b2'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 200
    lr = 0.003
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  00_efficientnet-b2  Created 


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [5]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.Flip(),
        A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),

        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.ElasticTransform(p=1.0),
            A.IAAPiecewiseAffine(p=1.0)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [6]:
train_df = pd.read_csv(config.root + 'train.csv')
train, valid = train_test_split(train_df, test_size=0.33, random_state=config.seed, shuffle=True, stratify=train_df[config.class_names])

train_transforms = compose([
    pre_transforms(config.size),
#     hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
#     hard_transforms()
])

train_dataset = PlantDataset(train, config, train_transforms)
valid_dataset = PlantDataset(valid, config, valid_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [7]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

model = get_model(config.model_name, config.num_classes)

Loaded pretrained weights for efficientnet-b2


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True, mode="max", factor=0.3)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

# end_lr = 0.1
# start_lr = 1e-7
# lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (lr_find_epochs * len(train_dataloader)))
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# scheduler_1 = LinearCyclicalScheduler(optimizer, "lr", start_value=0.01, end_value=0.005, cycle_size=60)
# scheduler_2 = CosineAnnealingScheduler(optimizer, "lr", start_value=0.05, end_value=0.001, cycle_size=60)
# combined_scheduler = ConcatScheduler(schedulers=[scheduler_1, scheduler_2], durations=[30, ])

In [9]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, criterion, optimizer, scheduler, device, config: ConfigExperiment):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    best_valid_loss = valid_loss
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}.pth")
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    self.counter = 0
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_auc_mean={best_valid_auc_mean}.pth")
                else:
                    self.counter += 1
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [10]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 12s
Training Results - Average Loss: 0.9544 | auc/_mean: 0.7295 | auc/healthy: 0.8006 | auc/multiple_diseases: 0.4911, auc/rust: 0.8038, auc/scab: 0.8224
Evaluating Results - Average Loss: 0.6672 | auc/_mean: 0.8466 | auc/healthy: 0.9258 | auc/multiple_diseases: 0.5785, auc/rust: 0.9422, auc/scab: 0.9401




Epoch: 02 | Time: 0m 12s
Training Results - Average Loss: 0.7159 | auc/_mean: 0.8479 | auc/healthy: 0.8816 | auc/multiple_diseases: 0.7278, auc/rust: 0.8871, auc/scab: 0.8952
Evaluating Results - Average Loss: 0.5892 | auc/_mean: 0.8703 | auc/healthy: 0.9294 | auc/multiple_diseases: 0.6761, auc/rust: 0.9432, auc/scab: 0.9325




Epoch: 03 | Time: 0m 13s
Training Results - Average Loss: 0.7058 | auc/_mean: 0.8525 | auc/healthy: 0.8953 | auc/multiple_diseases: 0.7464, auc/rust: 0.8938, auc/scab: 0.8746
Evaluating Results - Average Loss: 0.5959 | auc/_mean: 0.8674 | auc/healthy: 0.9279 | auc/multiple_diseases: 0.6754, auc/rust: 0.9353, auc/scab: 0.9312




Epoch: 04 | Time: 0m 13s
Training Results - Average Loss: 0.5928 | auc/_mean: 0.8947 | auc/healthy: 0.9168 | auc/multiple_diseases: 0.8273, auc/rust: 0.9125, auc/scab: 0.9222
Evaluating Results - Average Loss: 0.6029 | auc/_mean: 0.8625 | auc/healthy: 0.9329 | auc/multiple_diseases: 0.6498, auc/rust: 0.9353, auc/scab: 0.9319




Epoch: 05 | Time: 0m 13s
Training Results - Average Loss: 0.6176 | auc/_mean: 0.8822 | auc/healthy: 0.9129 | auc/multiple_diseases: 0.7916, auc/rust: 0.9157, auc/scab: 0.9087
Evaluating Results - Average Loss: 0.5996 | auc/_mean: 0.8643 | auc/healthy: 0.9285 | auc/multiple_diseases: 0.6597, auc/rust: 0.9364, auc/scab: 0.9325



...


Epoch: 22 | Time: 0m 13s
Training Results - Average Loss: 0.5130 | auc/_mean: 0.9243 | auc/healthy: 0.9299 | auc/multiple_diseases: 0.9095, auc/rust: 0.9272, auc/scab: 0.9307
Evaluating Results - Average Loss: 0.5681 | auc/_mean: 0.8734 | auc/healthy: 0.9341 | auc/multiple_diseases: 0.6816, auc/rust: 0.9381, auc/scab: 0.9397




Epoch: 23 | Time: 0m 13s
Training Results - Average Loss: 0.4772 | auc/_mean: 0.9277 | auc/healthy: 0.9431 | auc/multiple_diseases: 0.8939, auc/rust: 0.9392, auc/scab: 0.9348
Evaluating Results - Average Loss: 0.5662 | auc/_mean: 0.8735 | auc/healthy: 0.9342 | auc/multiple_diseases: 0.6816, auc/rust: 0.9378, auc/scab: 0.9403

Epoch    23: reducing learning rate of group 0 to 8.1000e-05.



Epoch: 24 | Time: 0m 13s
Training Results - Average Loss: 0.5158 | auc/_mean: 0.9221 | auc/healthy: 0.9246 | auc/multiple_diseases: 0.9014, auc/rust: 0.9387, auc/scab: 0.9235
Evaluating Results - Average Loss: 0.5704 | auc/_mean: 0.8729 | auc/healthy: 0.9335 | auc/multiple_diseases: 0.6831, auc/rust: 0.9364, auc/scab: 0.9388




Epoch: 25 | Time: 0m 13s
Training Results - Average Loss: 0.4864 | auc/_mean: 0.9270 | auc/healthy: 0.9318 | auc/multiple_diseases: 0.9042, auc/rust: 0.9444, auc/scab: 0.9276
Evaluating Results - Average Loss: 0.5697 | auc/_mean: 0.8731 | auc/healthy: 0.9341 | auc/multiple_diseases: 0.6833, auc/rust: 0.9369, auc/scab: 0.9382




Epoch: 26 | Time: 0m 13s
Training Results - Average Loss: 0.5226 | auc/_mean: 0.9189 | auc/healthy: 0.9344 | auc/multiple_diseases: 0.8820, auc/rust: 0.9287, auc/scab: 0.9305
Evaluating Results - Average Loss: 0.5679 | auc/_mean: 0.8734 | auc/healthy: 0.9345 | auc/multiple_diseases: 0.6835, auc/rust: 0.9377, auc/scab: 0.9381

EarlyStopping


In [11]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=15_auc_mean=0.8752601631798801.pth"))
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=config.patience, verbose=True, mode="max", factor=0.3)

Loaded pretrained weights for efficientnet-b2


In [12]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 31s
Training Results - Average Loss: 1.0728 | auc/_mean: 0.7834 | auc/healthy: 0.8403 | auc/multiple_diseases: 0.5745, auc/rust: 0.8864, auc/scab: 0.8326
Evaluating Results - Average Loss: 0.7967 | auc/_mean: 0.8686 | auc/healthy: 0.9772 | auc/multiple_diseases: 0.5533, auc/rust: 0.9797, auc/scab: 0.9643




Epoch: 02 | Time: 0m 30s
Training Results - Average Loss: 0.5564 | auc/_mean: 0.8771 | auc/healthy: 0.9461 | auc/multiple_diseases: 0.6802, auc/rust: 0.9550, auc/scab: 0.9273
Evaluating Results - Average Loss: 0.2672 | auc/_mean: 0.9108 | auc/healthy: 0.9919 | auc/multiple_diseases: 0.6748, auc/rust: 0.9960, auc/scab: 0.9807




Epoch: 03 | Time: 0m 33s
Training Results - Average Loss: 0.4426 | auc/_mean: 0.8740 | auc/healthy: 0.9609 | auc/multiple_diseases: 0.6098, auc/rust: 0.9715, auc/scab: 0.9541
Evaluating Results - Average Loss: 0.2817 | auc/_mean: 0.9299 | auc/healthy: 0.9930 | auc/multiple_diseases: 0.7437, auc/rust: 0.9935, auc/scab: 0.9893




Epoch: 04 | Time: 0m 34s
Training Results - Average Loss: 0.3536 | auc/_mean: 0.9022 | auc/healthy: 0.9813 | auc/multiple_diseases: 0.6804, auc/rust: 0.9729, auc/scab: 0.9740
Evaluating Results - Average Loss: 0.9688 | auc/_mean: 0.9149 | auc/healthy: 0.9542 | auc/multiple_diseases: 0.7277, auc/rust: 0.9923, auc/scab: 0.9853




Epoch: 05 | Time: 0m 33s
Training Results - Average Loss: 0.3884 | auc/_mean: 0.9059 | auc/healthy: 0.9761 | auc/multiple_diseases: 0.7112, auc/rust: 0.9703, auc/scab: 0.9662
Evaluating Results - Average Loss: 0.5718 | auc/_mean: 0.8669 | auc/healthy: 0.9338 | auc/multiple_diseases: 0.5678, auc/rust: 0.9858, auc/scab: 0.9803



...


Epoch: 16 | Time: 0m 31s
Training Results - Average Loss: 0.1347 | auc/_mean: 0.9801 | auc/healthy: 0.9957 | auc/multiple_diseases: 0.9354, auc/rust: 0.9932, auc/scab: 0.9961
Evaluating Results - Average Loss: 0.8873 | auc/_mean: 0.9082 | auc/healthy: 0.9829 | auc/multiple_diseases: 0.6843, auc/rust: 0.9816, auc/scab: 0.9840




Epoch: 17 | Time: 0m 30s
Training Results - Average Loss: 0.1877 | auc/_mean: 0.9636 | auc/healthy: 0.9950 | auc/multiple_diseases: 0.8745, auc/rust: 0.9926, auc/scab: 0.9924
Evaluating Results - Average Loss: 0.3372 | auc/_mean: 0.9060 | auc/healthy: 0.9915 | auc/multiple_diseases: 0.6447, auc/rust: 0.9957, auc/scab: 0.9922




Epoch: 18 | Time: 0m 30s
Training Results - Average Loss: 0.1924 | auc/_mean: 0.9579 | auc/healthy: 0.9952 | auc/multiple_diseases: 0.8524, auc/rust: 0.9932, auc/scab: 0.9909
Evaluating Results - Average Loss: 0.3266 | auc/_mean: 0.9219 | auc/healthy: 0.9891 | auc/multiple_diseases: 0.7148, auc/rust: 0.9943, auc/scab: 0.9893




Epoch: 19 | Time: 0m 30s
Training Results - Average Loss: 0.1446 | auc/_mean: 0.9711 | auc/healthy: 0.9970 | auc/multiple_diseases: 0.8998, auc/rust: 0.9946, auc/scab: 0.9932
Evaluating Results - Average Loss: 0.3795 | auc/_mean: 0.8838 | auc/healthy: 0.9899 | auc/multiple_diseases: 0.5622, auc/rust: 0.9951, auc/scab: 0.9881




Epoch: 20 | Time: 0m 30s
Training Results - Average Loss: 0.1802 | auc/_mean: 0.9697 | auc/healthy: 0.9929 | auc/multiple_diseases: 0.9022, auc/rust: 0.9925, auc/scab: 0.9911
Evaluating Results - Average Loss: 0.4255 | auc/_mean: 0.8942 | auc/healthy: 0.9791 | auc/multiple_diseases: 0.6208, auc/rust: 0.9947, auc/scab: 0.9821

Epoch    20: reducing learning rate of group 0 to 9.0000e-04.
EarlyStopping


In [44]:

class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, train_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [42]:
def predict(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [43]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=1_auc_mean=0.8706131482876178.pth"))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [45]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [46]:
y_probas = np.mean(result, axis=-1)

In [47]:
y_probas[:10]

array([[0.09819383, 0.3116633 , 0.1852319 , 0.40491095],
       [0.09934221, 0.13119815, 0.5209352 , 0.24852438],
       [0.00145755, 0.01327283, 0.002809  , 0.9824607 ],
       [0.9671133 , 0.00678661, 0.01244393, 0.0136562 ],
       [0.02429822, 0.02702083, 0.91830266, 0.0303783 ],
       [0.6532921 , 0.02755702, 0.12354162, 0.19560926],
       [0.8862263 , 0.03570914, 0.0465331 , 0.03153154],
       [0.0978264 , 0.04821354, 0.00504111, 0.84891903],
       [0.3122094 , 0.10618415, 0.03836362, 0.5432429 ],
       [0.27814376, 0.05621394, 0.5502282 , 0.11541412]], dtype=float32)

In [48]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.098194,0.311663,0.185232,0.404911
1,Test_1,0.099342,0.131198,0.520935,0.248524
2,Test_2,0.001458,0.013273,0.002809,0.982461
3,Test_3,0.967113,0.006787,0.012444,0.013656
4,Test_4,0.024298,0.027021,0.918303,0.030378


In [49]:
config.submission_file

'02_effb0_hard_aug.csv'